# Classification via logistic regression

In this set of exercises, you are going to implement logistic regression for email spam detection. We'll use the `Spambase` dataset from UCI Machine Learning Repository, you can find the description of the data from https://archive.ics.uci.edu/ml/datasets/spambase.  

In [245]:
using Plots;
Data = readdlm("../data/spambase.data", ',')

4601×58 Array{Float64,2}:
 0.0   0.64  0.64  0.0  0.32  0.0   0.0   …  0.0    3.756   61.0   278.0  1.0
 0.21  0.28  0.5   0.0  0.14  0.28  0.21     0.048  5.114  101.0  1028.0  1.0
 0.06  0.0   0.71  0.0  1.23  0.19  0.19     0.01   9.821  485.0  2259.0  1.0
 0.0   0.0   0.0   0.0  0.63  0.0   0.31     0.0    3.537   40.0   191.0  1.0
 0.0   0.0   0.0   0.0  0.63  0.0   0.31     0.0    3.537   40.0   191.0  1.0
 0.0   0.0   0.0   0.0  1.85  0.0   0.0   …  0.0    3.0     15.0    54.0  1.0
 0.0   0.0   0.0   0.0  1.92  0.0   0.0      0.0    1.671    4.0   112.0  1.0
 0.0   0.0   0.0   0.0  1.88  0.0   0.0      0.0    2.45    11.0    49.0  1.0
 0.15  0.0   0.46  0.0  0.61  0.0   0.3      0.022  9.744  445.0  1257.0  1.0
 0.06  0.12  0.77  0.0  0.19  0.32  0.38     0.0    1.729   43.0   749.0  1.0
 0.0   0.0   0.0   0.0  0.0   0.0   0.96  …  0.0    1.312    6.0    21.0  1.0
 0.0   0.0   0.25  0.0  0.38  0.25  0.25     0.0    1.243   11.0   184.0  1.0
 0.0   0.69  0.34  0.0  0.34  0.0   0.

Split the data into two sets: a training set and a testing set.

In [246]:
srand(1234)
y = Data[:,end]
y = 2*(y - 0.5)
X = Data[:,1:(end-1)]
n, d = size(X)
perm = randperm(n)
train_ratio = 0.8
n_train = round(Int, n*train_ratio )

Xtrain = X[ perm[1:n_train], : ]
Xtest = X[ perm[(n_train+1):end], : ]
ytrain = y[perm[1:n_train]]
ytest = y[ perm[(n_train+1):end] ]
;

Standardize the data to have zero mean.

In [247]:
mu = mean(Xtrain, 1)
sigma = zeros(d)
for i = 1:d
    sigma[i] = std(X[:,i])
    Xtrain[:,i] = (Xtrain[:,i] - mu[i]) / sigma[i]
    Xtest[:,i] = (Xtest[:,i] - mu[i]) / sigma[i]
end

The objective function for Logistic Regression with 2-norm regularization can be written as
$$ f(w) = \frac{1}{2} \|w\|^2 + C \sum_{i=1}^n \log(1 + \exp ( -y_i w^T x_i ) ). $$

## Question 1.
Complete the following code that evaluates the objective $f$.

In [248]:
function Obj(Xtrain, ytrain, w, C)
    sum = 0
    for i in 1:size(Xtrain,1)
        sum += log(1 + exp(-ytrain[i] * dot(w, Xtrain[i,:])))
    end
    return 0.5 * dot(w,w) + C*sum
end;

## Question 2.
Derive an expression for the gradient $\nabla f(w)$.

 $$\frac{\partial f(w)}{dw_j} = w_j + C \sum_{i=1}^n x_{ij}\frac{-y_i\exp(-y_i w^T x_i)}{1 + \exp ( -y_i w^T x_i )} $$
 $$Let \; r_i = \frac{-y_i\exp(-y_i w^T x_i)}{1 + \exp ( -y_i w^T x_i )} $$
 $$ \Rightarrow \nabla f(w) = w + CX^Tr$$

## Question 3.
Complete the following code that evaluates the gradient.

In [249]:
function Grad(Xtrain, ytrain, w, C)
    r = Array{Float64}(size(Xtrain,1))
    for i in 1:size(Xtrain,1)
        e = exp(-ytrain[i] * dot(w, Xtrain[i,:]))
        r[i] = -ytrain[i] * e / (1 + e) 
    end
    return w + C * Xtrain' * r
end;

## Question 4

Complete the function `gradient_descent`.

In [250]:
function gradient_descent(f, ∇f, w0, tol=1e-4, stepsize=1, maxIter=1000)

    # Initialization
    w = copy(w0)
    g = g0 = ∇f(w)
    norm_g0 = norm(g0)
    obj_new = pre_obj = f(w)
    iter_list = [0]
    obj_list = [pre_obj]
    μ=1e-5
    # Gradient Descent Iteration
    iter = 0
    while true
        
        @printf "it = %3d | f = %10.2e | ||∇f||  = %10.2e\n" iter obj_new norm(g)
        
        # Stop if the norm of the gradient is small relative to its start value
        if norm(g) < tol*norm_g0 || iter > maxIter
            break
        end
        α = stepsize
        while ( obj_new - f(w -α*g) < μ * α * dot(g,g) )
            α /= 2
        end
        
        w = w - α*g
        obj_new = f(w)
        g = ∇f(w)
        iter += 1
        
        push!(iter_list, iter)
        push!(obj_list, obj_new)
        
    end
    return w, iter_list, obj_list    
end;

## Question 5

Train the classifier using $C=1$. Report your test error and plot a figure where xlabel is the number of iteration and ylabel is the objective value.

In [251]:
# run the following code after you complete the functions
C = 1
f(x) = Obj(Xtrain, ytrain, x, C)
∇f(x) = Grad(Xtrain, ytrain, x, C)
w0 = zeros(size(Xtrain,2))
w, iter_list, obj_list = gradient_descent(f, ∇f, w0);
plot(iter_list, obj_list)

it =   0 | f =   2.55e+03 | ||∇f||  =   2.48e+03
it =   1 | f =   1.71e+03 | ||∇f||  =   5.10e+02
it =   2 | f =   1.69e+03 | ||∇f||  =   8.05e+02
it =   3 | f =   1.23e+03 | ||∇f||  =   3.78e+02
it =   4 | f =   9.57e+02 | ||∇f||  =   1.41e+02
it =   5 | f =   9.42e+02 | ||∇f||  =   2.36e+02
it =   6 | f =   9.23e+02 | ||∇f||  =   1.97e+02
it =   7 | f =   9.01e+02 | ||∇f||  =   1.72e+02
it =   8 | f =   9.00e+02 | ||∇f||  =   1.60e+02
it =   9 | f =   8.90e+02 | ||∇f||  =   1.55e+02
it =  10 | f =   8.69e+02 | ||∇f||  =   3.77e+01
it =  11 | f =   8.67e+02 | ||∇f||  =   3.86e+01
it =  12 | f =   8.66e+02 | ||∇f||  =   1.76e+01
it =  13 | f =   8.65e+02 | ||∇f||  =   4.14e+01
it =  14 | f =   8.64e+02 | ||∇f||  =   1.61e+01
it =  15 | f =   8.64e+02 | ||∇f||  =   2.06e+01
it =  16 | f =   8.64e+02 | ||∇f||  =   2.76e+01
it =  17 | f =   8.63e+02 | ||∇f||  =   1.07e+01
it =  18 | f =   8.63e+02 | ||∇f||  =   2.14e+01
it =  19 | f =   8.62e+02 | ||∇f||  =   9.35e+00
it =  20 | f =   8.6

it = 171 | f =   8.55e+02 | ||∇f||  =   4.80e+00
it = 172 | f =   8.55e+02 | ||∇f||  =   1.99e+00
it = 173 | f =   8.55e+02 | ||∇f||  =   4.95e+00
it = 174 | f =   8.55e+02 | ||∇f||  =   2.00e+00
it = 175 | f =   8.55e+02 | ||∇f||  =   5.12e+00
it = 176 | f =   8.55e+02 | ||∇f||  =   2.01e+00
it = 177 | f =   8.55e+02 | ||∇f||  =   5.29e+00
it = 178 | f =   8.55e+02 | ||∇f||  =   2.03e+00
it = 179 | f =   8.55e+02 | ||∇f||  =   2.49e+00
it = 180 | f =   8.55e+02 | ||∇f||  =   3.31e+00
it = 181 | f =   8.55e+02 | ||∇f||  =   4.66e+00
it = 182 | f =   8.55e+02 | ||∇f||  =   1.90e+00
it = 183 | f =   8.55e+02 | ||∇f||  =   4.80e+00
it = 184 | f =   8.55e+02 | ||∇f||  =   1.91e+00
it = 185 | f =   8.55e+02 | ||∇f||  =   4.96e+00
it = 186 | f =   8.55e+02 | ||∇f||  =   1.92e+00
it = 187 | f =   8.55e+02 | ||∇f||  =   2.35e+00
it = 188 | f =   8.55e+02 | ||∇f||  =   3.10e+00
it = 189 | f =   8.55e+02 | ||∇f||  =   4.36e+00
it = 190 | f =   8.54e+02 | ||∇f||  =   1.80e+00
it = 191 | f =   8.5

it = 341 | f =   8.54e+02 | ||∇f||  =   2.03e+00
it = 342 | f =   8.54e+02 | ||∇f||  =   7.68e-01
it = 343 | f =   8.54e+02 | ||∇f||  =   9.40e-01
it = 344 | f =   8.54e+02 | ||∇f||  =   1.24e+00
it = 345 | f =   8.54e+02 | ||∇f||  =   1.74e+00
it = 346 | f =   8.54e+02 | ||∇f||  =   7.13e-01
it = 347 | f =   8.54e+02 | ||∇f||  =   1.75e+00
it = 348 | f =   8.54e+02 | ||∇f||  =   7.09e-01
it = 349 | f =   8.54e+02 | ||∇f||  =   1.75e+00
it = 350 | f =   8.54e+02 | ||∇f||  =   7.04e-01
it = 351 | f =   8.54e+02 | ||∇f||  =   1.76e+00
it = 352 | f =   8.54e+02 | ||∇f||  =   7.00e-01
it = 353 | f =   8.54e+02 | ||∇f||  =   1.77e+00
it = 354 | f =   8.54e+02 | ||∇f||  =   6.96e-01
it = 355 | f =   8.54e+02 | ||∇f||  =   1.78e+00
it = 356 | f =   8.54e+02 | ||∇f||  =   6.92e-01
it = 357 | f =   8.54e+02 | ||∇f||  =   1.78e+00
it = 358 | f =   8.54e+02 | ||∇f||  =   6.88e-01
it = 359 | f =   8.54e+02 | ||∇f||  =   1.79e+00
it = 360 | f =   8.54e+02 | ||∇f||  =   6.84e-01
it = 361 | f =   8.5

it = 516 | f =   8.54e+02 | ||∇f||  =   2.51e-01
it = 517 | f =   8.54e+02 | ||∇f||  =   6.09e-01
it = 518 | f =   8.54e+02 | ||∇f||  =   2.49e-01
it = 519 | f =   8.54e+02 | ||∇f||  =   6.06e-01
it = 520 | f =   8.54e+02 | ||∇f||  =   2.46e-01


Print the training error and testing error

In [252]:
ypred = sign.(Xtest*w)
@printf "prediction accuracy on training set: %10.2e \n" sum(sign.(Xtrain*w) .== ytrain) / length(ytrain)
@printf "prediction accuracy on testing  set: %10.2e \n" sum(ypred .== ytest) / length(ypred)

prediction accuracy on training set:   9.30e-01 
prediction accuracy on testing  set:   9.05e-01 


## Question 6

Derive an expression for the Hessian matrix $\nabla^2 f(w)$.

Recall $$\frac{\partial f(w)}{dw_j} = w_j + C \sum_{i=1}^n x_{ij}\frac{-y_i\exp(-y_i w^T x_i)}{1 + \exp ( -y_i w^T x_i )} $$
$$\frac{\partial^2 f(w)}{\partial w_j^2} = 1 + c \sum_{i=1}^nx_{ij}^2 \frac{y_i^2\exp(-y_i w^T x_i)}{(1+\exp(-y_i w^T x_i))^2}$$
$$\frac{\partial^2 f(w)}{\partial w_jw_k} = c \sum_{i=1}^nx_{ij}x_{ik} \frac{y_i^2\exp(-y_i w^T x_i)}{(1+\exp(-y_i w^T x_i))^2}$$
Let $$r_i = \frac{y_i^2\exp(-y_i w^T x_i)}{(1+\exp(-y_i w^T x_i))^2}$$
And let $$ R\in \mathbb{R}^{n\times n} 
= \left[\begin{array}{c}
r_1 &        &       &        & 0 \\
    & \ddots &       &        &   \\
    &        &  r_i  &        &   \\
    &        &       & \ddots &   \\
0   &        &       &        & r_n
\end{array}\right]$$
$$\Rightarrow \nabla^2 f(w) = cX^TRX + I$$

## Question 7
Complete the following code that evaluates the Hessian matrix.

In [253]:
size_d = size(Xtrain,2)
size_n = size(Xtrain,1)

function Hessian(Xtrain, ytrain, w, c)
    r = Array{Float64}(size_n)
    for i in 1:size_n
        e = exp(-ytrain[i] * dot(w, Xtrain[i,:]))
        r[i] = (ytrain[i]^2) * e / (1 + e)^2
    end
    R = r .* eye(size_n)
    return c * Xtrain' * R * Xtrain + eye(size_d)
end

Hessian (generic function with 1 method)

## Question 8

Complete the function `newton_descent`. Note that we use fixed stepsize. Report your test error and plot a figure where xlabel is the number of iterations and ylabel is the objective value. Does the test error and objective value match with the result from gradient descent?

In [254]:
function newton_descent(f, ∇f, ∇²f, w0, tol=1e-4, maxIter = 1000)
    
    #Initialization
    w = copy(w0)
    g = g0 = ∇f(w)
    H = ∇²f(w)
    norm_g0 = norm(g0)
    obj_value = f(w)
    iter_list = [0]
    obj_list = [obj_value]
    
    iter = 0   
    while true
        
        @printf "it = %3d | f = %10.2e | ||∇f||  = %10.2e\n" iter obj_value norm(g)
        
        # Stop if the norm of the gradient is small relative to its start value
        if norm(g) < tol*norm_g0 || iter > maxIter
            break
        end
        iter += 1
        d = -H \ g
        w = w + d
        H = ∇²f(w)
        g = ∇f(w)
        
        #COMPLETE THE UPDATE RULE
        
        obj_value = f(w)
        
        push!(iter_list, iter)
        push!(obj_list, obj_value)
    
    end
    
    return w, iter_list, obj_list
end

newton_descent (generic function with 4 methods)

In [255]:
# Run the following code after you complete the functions
C = 1
f(x) = Obj(Xtrain, ytrain, x, C)
∇f(x) = Grad(Xtrain, ytrain, x, C)
hessian(x) = Hessian(Xtrain, ytrain, x, C)
w0 = zeros(size(Xtrain,2))
w, iter_list, obj_list = newton_descent(f, ∇f, hessian, w0);
plot(iter_list, obj_list)

it =   0 | f =   2.55e+03 | ||∇f||  =   2.48e+03
it =   1 | f =   1.38e+03 | ||∇f||  =   7.15e+02
it =   2 | f =   1.05e+03 | ||∇f||  =   2.72e+02
it =   3 | f =   9.13e+02 | ||∇f||  =   9.95e+01
it =   4 | f =   8.68e+02 | ||∇f||  =   3.27e+01
it =   5 | f =   8.55e+02 | ||∇f||  =   7.88e+00
it =   6 | f =   8.54e+02 | ||∇f||  =   8.68e-01
it =   7 | f =   8.54e+02 | ||∇f||  =   2.81e-02


Print the training error and testing error

In [256]:
ypred = sign.(Xtest*w);
@printf "prediction accuracy on training set: %10.2e \n" sum(sign.(Xtrain*w) .== ytrain) / length(ytrain)
@printf "prediction accuracy on testing  set: %10.2e \n" sum(ypred .== ytest) / length(ypred)

prediction accuracy on training set:   9.30e-01 
prediction accuracy on testing  set:   9.05e-01 


# Exercise 2

## Question 1

$$f(x) = \sum_{i=1}^m (\|(x- a_i\|^2 - d_i^2)^2$$
$$ Let \; r_i = \|x- a_i\|^2 - d_i^2 = x^Tx - 2a_i^Tx + a_i^Ta_i -d_i^2 $$
$$\Rightarrow \frac{\partial  r_i}{\partial x_j} = 2x_j - 2a_{ij} $$
$$\Rightarrow \nabla r_i = x - a_i^T$$
$$ A_k = \left[\begin{array}{c}
r_1(x_k)^T\\
\vdots\\
r_m(x_k)^T
\end{array}\right] =
\left[\begin{array}{c}
(x_k - a_1^T)^T \\
\vdots \\
(x_k - a_m^T)^T
\end{array}\right] =
\left[\begin{array}{c}
x_k^T - a_1^T \\
\vdots \\
x_k^T - a_m^T
\end{array}\right] = 
\left[\begin{array}{c}
X^T - A^T
\end{array}\right]$$

Where $X \in \mathbb{R}^{m \times n}$ and is simply $x_k^T$ stacked vertically $m$ times.

This gives us our Linearized problem

$$ \min f(x) = \|A_kx - b_k\|$$

Which can be solved by

$$ A_k^TA_kx = A_K^Tb_k $$

In order for a unqiue solution to exist at every iteration, $A_k^TA_k$ must be invertible, which is equivalent to saying $A_k$ must be full column rank.

Recall

$$A_k = 
\left[\begin{array}{c}
X^T - A^T
\end{array}\right]$$

$X^T$ is simply $x_k^T$ stacked vertically $m$ times so a unique solution will only exist when $A^T$ is full column rank, which occurs when $A$ is full row rank.

$$A = \left[\begin{array}{c}
- a_1 - \\
\vdots\\
- a_m - 
\end{array}\right]$$

and will be full row rank so long as each $a_i$ is linearly independent. Since $a_i \in \mathbb{R}^2$ we require that each $a_i$ not reside on the same line

## Question 2

In [303]:
srand(0)
A=randn(2,5)
x=randn(2)
d=sqrt(sum((A-x*ones(1,5)).^2))+0.05*randn(5)'
d=d'
m = size(A,2)
n = size(A,1)
A
A[1,:,]

LoadError: [91mBoundsError: attempt to access 2×5 Array{Float64,2} at index [1, Base.Slice(Base.OneTo(5)), 2][39m

In [275]:
function A_k(x, A, d)
    j = x' - A[:,1]'
    for i in 2:m
        r_i = x' - A[:,i]'
        j = vcat(j, r_i)
    end
    return j
end;

In [276]:
function r_k(x, A, d)
    r = Array{Float64}(m)
    for i in 1:m
        r[i] = norm(x - A[:,i])^2 - d[i]^2
    end
    return r
end;

In [281]:
function f(x, A, d)
    sum = 0
    for i in 1:m
        sum += (norm(x - A[:,1])^2 + d[i])^2
    end
    return sum
end;

In [312]:
function nlls_bt(f, R, Jacobian, x0, β=0.5, tol=1e-4, maxits=1000)
    xk = x0
    rk = R(xk)
    norm_rk0 = norm(rk)
    Ak = Jacobian(xk)
    obj_value = f(xk)
    iter_list = [0]
    obj_list = [obj_value]
    
    iter = 0   
    while true
        
        @printf "it = %3d | f = %10.2e | ||∇f||  = %10.2e\n" iter obj_value norm(rk)
        
        # Stop if the norm of the gradient is small relative to its start value
        if norm(rk) < tol*norm_rk0 || iter > maxits
            break
        end
        
        α = 0.5
        g = 2 * Ak' * Ak * xk - Ak' * ( Ak * xk - rk )
        while ( obj_value - f(xk -α*g) < β * α * dot(g,g) )
            α /= 2
        end
        
        iter += 1
        d = -Ak' * Ak \ Ak' * (Ak * xk - rk)
        xk = xk + d
        rk = R(xk)
        Ak = Jacobian(xk)
        
        #COMPLETE THE UPDATE RULE
        
        obj_value = f(xk)
        
        push!(iter_list, iter)
        push!(obj_list, obj_value)
    
    end
    
    return w, iter_list, obj_list
end

nlls_bt (generic function with 4 methods)

In [317]:
# Run the following code after you complete the functions
f(x) = f(x, A, d)
R(x) = r_k(x, A, d)
Jacobian(x) = A_k(x, A, d)
x0 = [1000 -500]'
x, iter_list, obj_list = newton_descent(f, R, Jacobian, x0);
plot(iter_list, obj_list)

it =   0 | f =   7.81e+12 | ||∇f||  =   2.80e+06
it =   1 | f =   7.35e+10 | ||∇f||  =   2.72e+05
it =   2 | f =   9.25e+08 | ||∇f||  =   3.06e+04
it =   3 | f =   9.04e+06 | ||∇f||  =   3.06e+03
it =   4 | f =   8.25e+04 | ||∇f||  =   2.70e+02


In [314]:
function gradient_descent(f, ∇f, w0, tol=1e-4, stepsize=1, maxIter=1000)

    # Initialization
    w = copy(w0)
    g = g0 = ∇f(w)
    norm_g0 = norm(g0)
    obj_new = pre_obj = f(w)
    iter_list = [0]
    obj_list = [pre_obj]
    μ=1e-5
    # Gradient Descent Iteration
    iter = 0
    while true
        
        @printf "it = %3d | f = %10.2e | ||∇f||  = %10.2e\n" iter obj_new norm(g)
        
        # Stop if the norm of the gradient is small relative to its start value
        if norm(g) < tol*norm_g0 || iter > maxIter
            break
        end
        α = stepsize
        while ( obj_new - f(w -α*g) < μ * α * dot(g,g) )
            α /= 2
        end
        
        w = w - α*g
        obj_new = f(w)
        g = ∇f(w)
        iter += 1
        
        push!(iter_list, iter)
        push!(obj_list, obj_new)
        
    end
    return w, iter_list, obj_list    
end;

In [315]:
function Grad(x, A)
    g = Array{Float64}(n)
    for i in 1:n
        sum = 0
        for j in 1:m
            sum += A[i,j]
        end
        g[i] = sum
    end
    return 4( x - g)
end;

In [316]:
# run the following code after you complete the functions
C = 1
f(x) = f(x, A, d)
∇f(x) = Grad(x, A)
x0 = [1000 -500]'
w, iter_list, obj_list = gradient_descent(f, ∇f, x0);
plot(iter_list, obj_list)

it =   0 | f =   7.81e+12 | ||∇f||  =   4.47e+03
it =   1 | f =   2.08e+02 | ||∇f||  =   1.04e-13
